In [1]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/raw/main/04-monitoring/data/results-gpt4o-mini.csv'

In [2]:
import pandas as pd
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [4]:
df = df.iloc[:300]

In [78]:
df

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp
...,...,...,...,...,...
295,An alternative way to load the data using the ...,Above users showed how to load the dataset dir...,8d209d6d,What is an alternative way to load the data us...,machine-learning-zoomcamp
296,You can directly download the dataset from Git...,Above users showed how to load the dataset dir...,8d209d6d,How can I directly download the dataset from G...,machine-learning-zoomcamp
297,You can fetch data for homework using the `req...,Above users showed how to load the dataset dir...,8d209d6d,Could you share a method to fetch data for hom...,machine-learning-zoomcamp
298,If the status code is 200 when downloading dat...,Above users showed how to load the dataset dir...,8d209d6d,What should I do if the status code is 200 whe...,machine-learning-zoomcamp


In [21]:
from sentence_transformers import SentenceTransformer,util

In [10]:
model = SentenceTransformer("multi-qa-mpnet-base-dot-v1")

/home/oleg/llm-workshop/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
answer_llm = df.iloc[0].answer_llm

In [18]:
answer_orig = df.iloc[0].answer_orig

In [14]:
vector = model.encode(answer_llm)

In [16]:
vector[0]

np.float32(-0.42244673)

### Q1
answer: np.float32(-0.42244673)

In [81]:
llm_and_orig_answers = df.to_dict(orient='records')

In [82]:
row = llm_and_orig_answers[0]

In [83]:
def compute_dot_similarity(row):
    answer_llm_emb = model.encode(row['answer_llm'])
    answer_orig_emb = model.encode(row['answer_orig'])

    return util.dot_score(answer_llm_emb, answer_orig_emb).cpu().tolist()


In [86]:
from tqdm import tqdm

In [91]:
evaluations = []

for row in tqdm(llm_and_orig_answers):
    dot_similarity = compute_dot_similarity(row)
    evaluations.append(dot_similarity)


100%|██████████| 300/300 [00:05<00:00, 53.97it/s]


In [92]:
evaluations

[[[17.5159969329834]],
 [[13.418407440185547]],
 [[25.313249588012695]],
 [[12.147418975830078]],
 [[18.747730255126953]],
 [[33.97039794921875]],
 [[30.25170135498047]],
 [[29.521587371826172]],
 [[35.27220153808594]],
 [[27.75175666809082]],
 [[32.3447151184082]],
 [[31.441844940185547]],
 [[36.38072967529297]],
 [[33.34051513671875]],
 [[30.60616111755371]],
 [[32.50304412841797]],
 [[29.674449920654297]],
 [[24.353464126586914]],
 [[20.132463455200195]],
 [[23.995466232299805]],
 [[30.880273818969727]],
 [[32.69243621826172]],
 [[30.04916763305664]],
 [[16.078166961669922]],
 [[31.79641342163086]],
 [[37.980018615722656]],
 [[20.83904266357422]],
 [[32.612857818603516]],
 [[38.89421081542969]],
 [[34.05183410644531]],
 [[28.26387596130371]],
 [[27.12483024597168]],
 [[23.975265502929688]],
 [[26.340139389038086]],
 [[18.658123016357422]],
 [[25.016395568847656]],
 [[21.10113525390625]],
 [[33.7267951965332]],
 [[29.340351104736328]],
 [[28.654495239257812]],
 [[29.60858154296875]],

In [93]:
import numpy as np

In [94]:
# Calculate the 75th percentile
percentile_75 = np.percentile(evaluations, 75)

In [95]:
percentile_75

np.float64(31.6743106842041)

### Q2
np.float64(31.6743106842041)

In [97]:
def compute_cosine_similarity(row):
    answer_llm_emb = model.encode(row['answer_llm'])
    norm = np.sqrt((answer_llm_emb * answer_llm_emb).sum())
    norm_llm = answer_llm_emb / norm

    answer_orig_emb = model.encode(row['answer_orig'])
    norm = np.sqrt((answer_orig_emb * answer_orig_emb).sum())
    norm_orig = answer_orig_emb / norm


    return util.dot_score(norm_llm, norm_orig).cpu().tolist()


In [98]:
evaluations_normalized = []

for row in tqdm(llm_and_orig_answers):
    cosine_similarity = compute_cosine_similarity(row)
    evaluations_normalized.append(cosine_similarity)


100%|██████████| 300/300 [00:05<00:00, 53.04it/s]


In [99]:
cosine_percentile_75 = np.percentile(evaluations_normalized, 75)

In [100]:
cosine_percentile_75

np.float64(0.8362348079681396)

### Q3
np.float64(0.8362348079681396)

In [103]:
from rouge import Rouge
rouge_scorer = Rouge()

In [107]:
scores = rouge_scorer.get_scores(llm_and_orig_answers[10]['answer_llm'], llm_and_orig_answers[10]['answer_orig'])[0]

In [108]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

### Q4
  'f': 0.45454544954545456},